In [153]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [154]:
train_file_path = "../data/raw/train.csv"
test_file_path = "../data/raw/test.csv"
variable_notes_path = "../data/external/variable_notes.csv"

train = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)
variable_notes = pd.read_csv(variable_notes_path)

train.drop("Id", axis=1, inplace=True)

In [155]:
# Process variable notes for category mapping 
category_mapping = variable_notes.set_index("Column")["category"].fillna("Unknown").to_dict()  
from collections import defaultdict  
category_columns = defaultdict(list)  
for col, cat in category_mapping.items():  
    category_columns[cat].append(col)  

# Print feature categories and associated columns
print("\nFeature Categories and Associated Columns:")
for category, cols in category_columns.items():
    print(f"\n{category}:\n  - " + "\n  - ".join(cols))  


Feature Categories and Associated Columns:

basement:
  - BsmtFinSF1
  - BsmtUnfSF
  - BsmtCond
  - BsmtExposure
  - BsmtFinType1
  - BsmtFinType2
  - BsmtFullBath
  - BsmtHalfBath
  - BsmtQual
  - BsmtFinSF2
  - TotalBsmtSF

bath:
  - FullBath
  - HalfBath

exterior:
  - Exterior1st
  - Exterior2nd
  - MasVnrType
  - ExterCond
  - ExterQual
  - MasVnrArea
  - PavedDrive
  - WoodDeckSF

fireplace:
  - Fireplaces

garage:
  - GarageCond
  - GarageFinish
  - GarageQual
  - GarageType
  - GarageYrBlt
  - GarageArea
  - GarageCars

general:
  - MSSubClass
  - HouseStyle
  - MSZoning
  - Functional
  - MiscVal
  - Neighborhood
  - OverallCond
  - OverallQual
  - Utilities
  - YearRemodAdd

high missing:
  - FireplaceQu
  - Alley
  - Fence
  - MiscFeature
  - PoolQC

interior:
  - BldgType
  - CentralAir
  - Heating
  - Electrical
  - HeatingQC
  - 1stFlrSF
  - 2ndFlrSF
  - BedroomAbvGr
  - GrLivArea
  - LowQualFinSF
  - TotRmsAbvGrd

kitchen:
  - KitchenQual
  - KitchenAbvGr

lot:
  - LotC

In [156]:
def display_column_info(df, column):
    """Display type and unique values of a column in the dataframe."""
    if column not in df.columns:
        print(f"Column '{column}' not found in dataframe.")
        return
    
    values = df[column].unique()
    print(f"\nColumn: {column}\nType: {df[column].dtype}\nUnique Values: " +
          (", ".join(map(str, values[:10])) + "..." if len(values) > 10 else ", ".join(map(str, values))))
    print("-" * 50)

In [158]:
from IPython.display import display, HTML

def display_category_info(df, category):
    """Display details and correlations for variables within a specified category in a prettier, more readable format."""
    if category not in category_columns:
        display(HTML(f"<p style='color:red; font-weight:bold;'>Category '{category}' not found.</p>"))
        return

    cols = category_columns[category]
    
    # Create a formatted header using HTML
    header_html = f"""
    <div style="border: 2px solid #444; padding: 10px; margin-bottom: 10px; background-color: #f2f2f2;">
        <h2 style="margin: 0; color: #444;">Category: {category.upper()}</h2>
    </div>
    """
    display(HTML(header_html))
    
    # Build a DataFrame listing each column with its correlation to 'SalePrice'
    corr_values = []
    if "SalePrice" in df.columns:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        for col in cols:
            if col in numeric_cols:
                corr = df[col].corr(df["SalePrice"])
                corr_values.append(corr)
            else:
                corr_values.append(np.nan)
    else:
        corr_values = [np.nan] * len(cols)
    
    cols_corr_df = pd.DataFrame({
        "Column": cols,
        "Correlation with SalePrice": [f"{val:.3f}" if pd.notnull(val) else "--" for val in corr_values]
    })
    
    styled_table = cols_corr_df.style.set_table_styles(
        [{
            'selector': 'th',
            'props': [('background-color', '#f7f7f7'),
                      ('color', '#333'),
                      ('font-weight', 'bold'),
                      ('padding', '5px')]
        },
        {
            'selector': 'td',
            'props': [('padding', '5px')]
        }]
    ).set_caption("Columns and Correlations")
    
    display(styled_table)
    
    # Display detailed information for each column
    detail_header = """
    <h3 style="color:#444; border-bottom:1px solid #ccc; margin-bottom:10px;">
        Detailed Column Information:
    </h3>
    """
    display(HTML(detail_header))
    
    for col in cols:
        sub_header = f"""
        <h4 style="color:#666; margin-bottom:0;">{col}</h4>
        <hr style="border:0; border-bottom:1px solid #ccc; margin:5px 0 10px;">
        """
        display(HTML(sub_header))
        display_column_info(df, col)


In [159]:
categories_to_display = ["basement", "bath", "exterior", "fireplace", "garage",
                         "general", "interior", "kitchen", "lot", "overall",
                         "pool", "porch", "roof", "sale_info"]  

for cat in categories_to_display:  
    display_category_info(train, cat)  

,Column,Correlation with SalePrice
0,BsmtFinSF1,0.386
1,BsmtUnfSF,0.214
2,BsmtCond,--
3,BsmtExposure,--
4,BsmtFinType1,--
5,BsmtFinType2,--
6,BsmtFullBath,0.227
7,BsmtHalfBath,-0.017
8,BsmtQual,--
9,BsmtFinSF2,-0.011



Column: BsmtFinSF1
Type: int64
Unique Values: 706, 978, 486, 216, 655, 732, 1369, 859, 0, 851...
--------------------------------------------------



Column: BsmtUnfSF
Type: int64
Unique Values: 150, 284, 434, 540, 490, 64, 317, 216, 952, 140...
--------------------------------------------------



Column: BsmtCond
Type: object
Unique Values: TA, Gd, nan, Fa, Po
--------------------------------------------------



Column: BsmtExposure
Type: object
Unique Values: No, Gd, Mn, Av, nan
--------------------------------------------------


...

,Column,Correlation with SalePrice
0,MoSold,0.046
1,SaleCondition,--
2,SaleType,--
3,YrSold,-0.029



Column: MoSold
Type: int64
Unique Values: 2, 5, 9, 12, 10, 8, 11, 4, 1, 7...
--------------------------------------------------



Column: SaleCondition
Type: object
Unique Values: Normal, Abnorml, Partial, AdjLand, Alloca, Family
--------------------------------------------------



Column: SaleType
Type: object
Unique Values: WD, New, COD, ConLD, ConLI, CWD, ConLw, Con, Oth
--------------------------------------------------



Column: YrSold
Type: int64
Unique Values: 2008, 2007, 2006, 2009, 2010
--------------------------------------------------
